# Model Training and Evaluation

---

## Setup


In [ ]:
# Move up to project root directory (parent directory) for module imports
import os

os.chdir("../")

# Current working directory should now be project root
print("Current working directory:", os.getcwd())

In [ ]:
# Imports

import torch
from transformers import set_seed

from src.dataset import CocoDataset
from src.models import ImageCaptioningModel, TransformerMappingNetwork
from src.train import train

In [ ]:
# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
# Seed
SEED = 42
set_seed(SEED)  # Helper function that sets the seed in all relevant libraries

---

## Prepare Datasets


In [ ]:
MAX_CAPTION_LENGTH = 50
DATA_DIR = "coco_data/"
EMBEDDINGS_PATH = DATA_DIR + "embeddings/"
ANNOTATIONS_PATH = DATA_DIR + "annotations/"
CHECKPOINTS_PATH = "checkpoints/"
VAL_PATH = DATA_DIR + "val2017/"  # For FiftyOne visualization

In [ ]:
# Training Dataset
train_dataset = CocoDataset(
    embeddings_path=EMBEDDINGS_PATH + "train_clip_embeddings.pt",
    annotations_path=ANNOTATIONS_PATH + "captions_train2017.json",
    max_length=MAX_CAPTION_LENGTH,
    normalize_embeddings=False,  # `.pt` files already contain normalized embeddings
)

# Validation Dataset
val_dataset = CocoDataset(
    embeddings_path=EMBEDDINGS_PATH + "val_clip_embeddings.pt",
    annotations_path=ANNOTATIONS_PATH + "captions_val2017.json",
    max_length=MAX_CAPTION_LENGTH,
    normalize_embeddings=False,
)

# Note: No test dataset as test annotations are not publicly available

---

## Prepare Model


In [ ]:
# Transformer Mapping Network Params
EMBED_DIM = 512  # Embedding dimension
GPT_DIM = 768  # GPT-2 embedding dimension
PREFIX_LENGTH = 40
HIDDEN_LENGTH = 40
LAYER_TYPE = "transformer"

# Image Captioning Model Params
FREEZE_GPT_WEIGHTS = True  # Whether to fine-tune GPT-2 alongside the mapping network

In [ ]:
# Models
mapping_network = TransformerMappingNetwork(
    embed_dim=EMBED_DIM,
    gpt_dim=GPT_DIM,
    prefix_length=PREFIX_LENGTH,
    hidden_length=HIDDEN_LENGTH,
    layer_type=LAYER_TYPE,
)

model = ImageCaptioningModel(
    mapping_network=mapping_network,
    freeze_gpt_weights=FREEZE_GPT_WEIGHTS,
).to(DEVICE)

print(model)

---

## Train Model (with Validation Evaluation)


In [ ]:
# Training Params
TRAIN_BATCH_SIZE = 64
NUM_EPOCHS = 1
NUM_WORKERS = 4
LEARNING_RATE = 2e-5
NUM_WARMUP_STEPS = 0
SAVE_EVERY_EPOCH = 1

# Validation Params
EVAL_EVERY_EPOCH = 1
EVAL_BATCH_SIZE = 64
EVAL_MAX_CAPTION_LENGTH = MAX_CAPTION_LENGTH
EVAL_TEMPERATURE = 1.0
EVAL_TOP_P = 0.9

In [ ]:
# Train image captioning model with validation evaluation
train_history = train(
    # Training params
    train_dataset=train_dataset,
    model=model,
    batch_size=TRAIN_BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    num_workers=NUM_WORKERS,
    learning_rate=LEARNING_RATE,
    num_warmup_steps=NUM_WARMUP_STEPS,
    save_every_epoch=SAVE_EVERY_EPOCH,
    device=DEVICE,
    outputs_dir=CHECKPOINTS_PATH,
    # Eval params
    val_dataset=val_dataset,
    val_annotations_path=ANNOTATIONS_PATH + "val_split.json",
    eval_every_epoch=EVAL_EVERY_EPOCH,
    eval_batch_size=EVAL_BATCH_SIZE,
    eval_max_length=EVAL_MAX_CAPTION_LENGTH,
    eval_temperature=EVAL_TEMPERATURE,
    eval_top_p=EVAL_TOP_P,
)

print(
    f"\nBest validation CIDEr: {train_history['best_val_cider']:.4f} at epoch {train_history['best_epoch']}"
)

In [ ]:
train_history